#IMPORTS


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# librairies générales
import pandas as pd
import re
from tabulate import tabulate
import time
import numpy as np
import pickle
import string
import base64
import sys

# librairie affichage
import matplotlib.pyplot as plt
import seaborn as sns

# librairies scikit learn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# librairies des classifiers utilisés
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#!pip install ydata-profiling

  Using cached visions-0.7.4-py3-none-any.whl.metadata (5.9 kB)
Using cached visions-0.7.4-py3-none-any.whl (102 kB)
  Attempting uninstall: visions
    Found existing installation: visions 0.8.1
    Uninstalling visions-0.8.1:
      Successfully uninstalled visions-0.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires visions[type_image_path]<0.8.2,>=0.7.5, but you have visions 0.7.4 which is incompatible.
  Using cached visions-0.8.1-py3-none-any.whl.metadata (11 kB)
Using cached visions-0.8.1-py3-none-any.whl (105 kB)
  Attempting uninstall: visions
    Found existing installation: visions 0.7.4
    Uninstalling visions-0.7.4:
      Successfully uninstalled visions-0.7.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

In [ ]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/TER'
sys.path.append(my_local_drive)
%cd $my_local_drive
%pwd
import sys
sys.path.append("/content/gdrive/My Drive/Colab Notebooks/projet_ML")
from MyNLPUtilities import *

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/TER


#Exploration de la base de données

##Observation et mise en place



In [ ]:
#Lire le fichier excel
df = pd.read_excel("all_data.xlsx", engine="openpyxl")

Dans cette partie, nous allons prendre en compte le test "ISAC_V1". Dans une autre partie, nous parlerons des autres types de tests, tels que l’ISAC_V2 et l’ALEX.

In [ ]:
# Filtrer les lignes où Chip_type == "ISAC_V1"
V1_df = df[df["Chip_Type"] == "ISAC_V1"]
V1_df.head()

,Unnamed: 0,Chip_Code,Chip_Type,Chip_Image_Name,Age,Gender,Blood_Month_sample,French_Residence_Department,French_Region,Rural_or_urban_area,...,Tyr_p_2,Ulm_c,Urt_d,Vac_m,Ves_v,Ves_v_1,Vit_v_1,Xip_g_1,Zea_m,Zea_m_14
0,XPW0007,XPW0007,ISAC_V1,NaN,9.0,M,3.0,32,11,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,XPW0011,XPW0011,ISAC_V1,NaN,25.0,F,5.0,31,11,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,XPW0013,XPW0013,ISAC_V1,NaN,59.0,F,5.0,82,11,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,XPW0017,XPW0017,ISAC_V1,NaN,49.0,F,5.0,33,10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,XPW0018,XPW0018,ISAC_V1,NaN,9.0,F,5.0,33,10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
V1_df.describe()

,Age,Blood_Month_sample,French_Residence_Department,French_Region,Rural_or_urban_area,Allergy_Present,Severe_Allergy,Respiratory_Allergy,Food_Allergy,Venom_Allergy,...,Tyr_p_2,Ulm_c,Urt_d,Vac_m,Ves_v,Ves_v_1,Vit_v_1,Xip_g_1,Zea_m,Zea_m_14
count,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.00000,2351.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,21.677584,6.405359,651.428328,11.150574,6.992344,0.523182,0.340281,0.809443,0.64866,0.078265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,17.922809,3.634977,455.931904,2.200648,3.543520,0.499569,0.473904,0.392824,0.47749,0.268645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,8.000000,3.000000,77.000000,10.000000,9.000000,0.000000,0.000000,1.000000,0.00000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,15.000000,6.000000,999.000000,11.000000,9.000000,1.000000,0.000000,1.000000,1.00000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,32.000000,10.000000,999.000000,13.000000,9.000000,1.000000,1.000000,1.000000,1.00000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,102.000000,12.000000,999.000000,14.000000,9.000000,1.000000,1.000000,1.000000,1.00000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Comme on peut le voir, il y a beaucoup de valeurs manquantes. C’est pourquoi nous supprimons les colonnes contenant des allergènes sans valeur.

In [ ]:
V1 = V1_df.dropna(axis=1)
V1.describe()

,Age,Blood_Month_sample,French_Residence_Department,French_Region,Rural_or_urban_area,Allergy_Present,Severe_Allergy,Respiratory_Allergy,Food_Allergy,Venom_Allergy,...,Pla_l_1,Pol_d_5,Pru_p_1,Pru_p_3,Sal_k_1,Ses_i_1,Tri_a_14,Tri_a_19.0101,Tri_a_aA_TI,Ves_v_5
count,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.00000,2351.000000,...,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000
mean,21.677584,6.405359,651.428328,11.150574,6.992344,0.523182,0.340281,0.809443,0.64866,0.078265,...,3.186023,0.077044,2.587427,0.677465,0.268558,0.578669,0.243092,0.055751,0.080502,0.099000
std,17.922809,3.634977,455.931904,2.200648,3.543520,0.499569,0.473904,0.392824,0.47749,0.268645,...,14.734573,0.567807,11.089689,3.595603,2.967596,4.559418,3.033733,0.855691,0.808743,0.804392
min,0.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,3.000000,77.000000,10.000000,9.000000,0.000000,0.000000,1.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15.000000,6.000000,999.000000,11.000000,9.000000,1.000000,0.000000,1.000000,1.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,32.000000,10.000000,999.000000,13.000000,9.000000,1.000000,1.000000,1.000000,1.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,102.000000,12.000000,999.000000,14.000000,9.000000,1.000000,1.000000,1.000000,1.00000,1.000000,...,202.180000,12.910000,172.490000,105.910000,77.140000,104.550000,88.180000,34.950000,21.230000,17.900000


##Pandas Profiling

Dans cette partie, nous allons utiliser pandas_profiling afin qu’il génère quelques résultats et statistiques.

In [ ]:
#from ydata_profiling import ProfileReport

In [ ]:
#report = ProfileReport(V1, title="V1 Rapport", explorative=True, minimal=True)
#report.to_file('V1_rapport.html')

/usr/local/lib/python3.11/dist-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 176/176 [00:02<00:00, 59.51it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

##Ingenierie des données et reformulation

In [ ]:
from tqdm import tqdm
import pandas as pd

def expand_custom_one_hot(df, value_lists: dict):
    df = df.copy()
    for col in tqdm(value_lists.keys()):
        valid_values = value_lists[col]

        for val in valid_values:
            df.loc[:, f"{col}_{val}"] = 0

        split_values = df[col].astype(str).str.split(r"[,.]", expand=True)

        # remplir les colonnes
        for idx, row in split_values.iterrows():
            for val in row:
                if val and val.strip().isdigit():
                    val_int = int(val.strip())
                    if val_int in valid_values:
                        df.loc[idx, f"{col}_{val_int}"] = 1
    return df

In [ ]:
value_lists = {
    'Treatment_of_rhinitis': [0, 1, 2, 3, 4, 9],
    'Treatment_of_athsma': [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11],
    'General_cofactors': [0,1,2,3,4,5,6,7,8,9,10,11,12],
    'Age_of_onsets': [0,1,2,3,4,5,6,9],
    'Treatment_of_atopic_dematitis': [0,1,2,3,4,5,6,9],
    'ARIA_(rhinitis)': [0,1,2,3,4,5,9],
    'GINA_(asthma)': [0,1,2,3,4,5,9]
}

V1 = expand_custom_one_hot(V1, value_lists)
V1 = V1.drop('Type_of_Respiratory_Allergy', axis=1)
V1 = V1.drop('Type_of_Food_Allergy', axis=1)
V1 = V1.drop('Type_of_Venom_Allergy', axis=1)
V1 = V1.drop(columns=[
    'Treatment_of_rhinitis',
    'Treatment_of_athsma',
    'General_cofactors',
    'Age_of_onsets',
    'Treatment_of_atopic_dematitis',
    'ARIA_(rhinitis)',
    'GINA_(asthma)',
    'Food_List',
    'Oral_food_challenge',
    'Symptoms_per_food'
])

100%|██████████| 7/7 [00:07<00:00,  1.06s/it]


Ne pas executer cette commande ( les models SVM, et Logistic regression ne fonctionne pas sur des NAN)

In [ ]:
"""
V1[[
    'Rural_or_urban_area', 'Skin_Symptoms', 'Conjunctivitis', 'Oral_Syndrom',
    'Cardiovascular_symptoms', 'Respiratory_symptoms', 'Gastrointestinal_symptoms',
    'History_of_food_anaphylaxis', 'First_degree_family_history_of_atopy',
    'History_of_atopic_dematitis', 'History_of_mast_cell_disorders',
    'History_of_hymenoptera_venom_anaphylaxis', 'Mammalian_meat', 'Food_polyallergies'
]] = V1[[
    'Rural_or_urban_area', 'Skin_Symptoms', 'Conjunctivitis', 'Oral_Syndrom',
    'Cardiovascular_symptoms', 'Respiratory_symptoms', 'Gastrointestinal_symptoms',
    'History_of_food_anaphylaxis', 'First_degree_family_history_of_atopy',
    'History_of_atopic_dematitis', 'History_of_mast_cell_disorders',
    'History_of_hymenoptera_venom_anaphylaxis', 'Mammalian_meat', 'Food_polyallergies'
]].replace({9: np.nan})

"""

In [ ]:
import pandas as pd

# categories des allergenes
allergenes = {
    "Pollens": [
        "Aln_g_1", "Amb_a_1", "Art_v_1", "Art_v_3", "Bet_v_1", "Bet_v_2", "Bet_v_4",
        "Che_a_1", "Cry_j_1", "Cup_a_1", "Cyn_d_1", "Mer_a_1", "Ole_e_1", "Ole_e_7",
        "Ole_e_9", "Par_j_2", "Phl_p_1", "Phl_p_2", "Phl_p_4", "Phl_p_5", "Phl_p_6",
        "Phl_p_7", "Phl_p_11", "Phl_p_12", "Pla_a_1", "Pla_a_2", "Pla_a_3", "Pla_l_1"
    ],
    "Moisissures": [
        "Alt_a_1", "Alt_a_6", "Asp_f_1", "Asp_f_3", "Asp_f_6", "Cla_h_8",
        "Pen_m_1", "Pen_m_2", "Pen_m_4"
    ],
    "Animaux": [
        "Bla_g_1", "Bla_g_2", "Bla_g_5", "Bla_g_7", "Blo_t_5", "Can_f_1", "Can_f_2",
        "Can_f_3", "Can_f_5", "Equ_c_1", "Equ_c_3", "Fel_d_1", "Fel_d_2", "Fel_d_4",
        "Mus_m_1", "Der_f_1", "Der_f_2", "Der_p_1", "Der_p_2", "Der_p_10", "Lep_d_2"
    ],
    "Alimentaires": [
        "Act_d_1", "Act_d_2", "Act_d_5", "Act_d_8", "Ana_o_2", "Ani_s_1", "Ani_s_3",
        "Api_g_1", "Ara_h_1", "Ara_h_2", "Ara_h_3", "Ara_h_6", "Ara_h_8", "Ara_h_9",
        "Ber_e_1", "Bos_d_4", "Bos_d_5", "Bos_d_6", "Bos_d_8", "Bos_d_Lactoferrin",
        "Cor_a_1.0101", "Cor_a_1.0401", "Cor_a_8", "Cor_a_9", "Fag_e_2", "Gad_c_1",
        "Gal_d_1", "Gal_d_2", "Gal_d_3", "Gal_d_5", "Gly_m_4", "Gly_m_5", "Gly_m_6",
        "Jug_r_1", "Jug_r_2", "Jug_r_3", "Mal_d_1", "Pru_p_1", "Pru_p_3", "Sal_k_1",
        "Ses_i_1", "Tri_a_14", "Tri_a_19.0101", "Tri_a_aA_TI"
    ],
    "Venins": ["Api_m_1", "Api_m_4", "Pol_d_5", "Ves_v_5"],
    "Glycanes": ["Hev_b_1", "Hev_b_3", "Hev_b_5", "Hev_b_6.01", "Hev_b_8", "MUXF3"]
}


#mettre les colonnes min et max et moyennes
for type_allergene, colonnes in allergenes.items():
    colonnes_presentes = [col for col in colonnes if col in V1.columns]
    V1[f"Moyenne_{type_allergene}"] = V1[colonnes_presentes].mean(axis=1)
    V1[f"Max_{type_allergene}"] = V1[colonnes_presentes].max(axis=1)
    V1[f"Min_{type_allergene}"] = V1[colonnes_presentes].min(axis=1)

In [ ]:
V1.describe()

,Age,Blood_Month_sample,French_Residence_Department,French_Region,Rural_or_urban_area,Allergy_Present,Severe_Allergy,Respiratory_Allergy,Food_Allergy,Venom_Allergy,...,Min_Animaux,Moyenne_Alimentaires,Max_Alimentaires,Min_Alimentaires,Moyenne_Venins,Max_Venins,Min_Venins,Moyenne_Glycanes,Max_Glycanes,Min_Glycanes
count,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.00000,2351.000000,...,2351.0,2351.000000,2351.000000,2351.0,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000
mean,21.677584,6.405359,651.428328,11.150574,6.992344,0.523182,0.340281,0.809443,0.64866,0.078265,...,0.0,0.881073,13.238426,0.0,0.052087,0.162633,0.000094,0.544665,2.939137,0.000060
std,17.922809,3.634977,455.931904,2.200648,3.543520,0.499569,0.473904,0.392824,0.47749,0.268645,...,0.0,2.095343,26.090574,0.0,0.318630,0.917593,0.004537,2.274002,12.262601,0.002887
min,0.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,3.000000,77.000000,10.000000,9.000000,0.000000,0.000000,1.000000,0.00000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15.000000,6.000000,999.000000,11.000000,9.000000,1.000000,0.000000,1.000000,1.00000,0.000000,...,0.0,0.093409,2.220000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,32.000000,10.000000,999.000000,13.000000,9.000000,1.000000,1.000000,1.000000,1.00000,0.000000,...,0.0,0.805568,13.640000,0.0,0.000000,0.000000,0.000000,0.056667,0.305000,0.000000
max,102.000000,12.000000,999.000000,14.000000,9.000000,1.000000,1.000000,1.000000,1.00000,1.000000,...,0.0,26.455000,227.440000,0.0,7.000000,17.900000,0.220000,41.395000,238.190000,0.140000


In [ ]:
col_allergenes = [
        "Aln_g_1", "Amb_a_1", "Art_v_1", "Art_v_3", "Bet_v_1", "Bet_v_2", "Bet_v_4",
        "Che_a_1", "Cry_j_1", "Cup_a_1", "Cyn_d_1", "Mer_a_1", "Ole_e_1", "Ole_e_7",
        "Ole_e_9", "Par_j_2", "Phl_p_1", "Phl_p_2", "Phl_p_4", "Phl_p_5", "Phl_p_6",
        "Phl_p_7", "Phl_p_11", "Phl_p_12", "Pla_a_1", "Pla_a_2", "Pla_a_3", "Pla_l_1",
        "Alt_a_1", "Alt_a_6", "Asp_f_1", "Asp_f_3", "Asp_f_6", "Cla_h_8",
        "Pen_m_1", "Pen_m_2", "Pen_m_4",
        "Bla_g_1", "Bla_g_2", "Bla_g_5", "Bla_g_7", "Blo_t_5", "Can_f_1", "Can_f_2",
        "Can_f_3", "Can_f_5", "Equ_c_1", "Equ_c_3", "Fel_d_1", "Fel_d_2", "Fel_d_4",
        "Mus_m_1", "Der_f_1", "Der_f_2", "Der_p_1", "Der_p_2", "Der_p_10", "Lep_d_2",
        "Act_d_1", "Act_d_2", "Act_d_5", "Act_d_8", "Ana_o_2", "Ani_s_1", "Ani_s_3",
        "Api_g_1", "Ara_h_1", "Ara_h_2", "Ara_h_3", "Ara_h_6", "Ara_h_8", "Ara_h_9",
        "Ber_e_1", "Bos_d_4", "Bos_d_5", "Bos_d_6", "Bos_d_8", "Bos_d_Lactoferrin",
        "Cor_a_1.0101", "Cor_a_1.0401", "Cor_a_8", "Cor_a_9", "Fag_e_2", "Gad_c_1",
        "Gal_d_1", "Gal_d_2", "Gal_d_3", "Gal_d_5", "Gly_m_4", "Gly_m_5", "Gly_m_6",
        "Jug_r_1", "Jug_r_2", "Jug_r_3", "Mal_d_1", "Pru_p_1", "Pru_p_3", "Sal_k_1",
        "Ses_i_1", "Tri_a_14", "Tri_a_19.0101", "Tri_a_aA_TI", "Api_m_1", "Api_m_4", "Pol_d_5", "Ves_v_5",
        "Hev_b_1", "Hev_b_3", "Hev_b_5", "Hev_b_6.01", "Hev_b_8", "MUXF3"]
V1.drop(col_allergenes, axis=1, inplace=True)

Il y a des valeurs ou Allergy_Present = 0 et d'autre Type d'allergy sont presente c pour cela j'ai nettoyer en mettant tous les autres types d'allergie egale à 0 si Allergy_Present = 0

In [ ]:
V1.loc[V1["Allergy_Present"] == 0, ["Respiratory_Allergy", "Food_Allergy", "Venom_Allergy"]] = 0
V1.loc[V1["Respiratory_Allergy"] == 0, [
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA"
]] = 0

V1.loc[V1["Food_Allergy"] == 0, [
    "Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts"
]] = 0

V1.loc[V1["Venom_Allergy"] == 0, ["Type_of_Venom_Allergy_ATCD_Venom",
    "Type_of_Venom_Allergy_IGE_Venom"]] = 0

In [ ]:
V1.describe()

,Age,Blood_Month_sample,French_Residence_Department,French_Region,Rural_or_urban_area,Allergy_Present,Severe_Allergy,Respiratory_Allergy,Food_Allergy,Venom_Allergy,...,Min_Animaux,Moyenne_Alimentaires,Max_Alimentaires,Min_Alimentaires,Moyenne_Venins,Max_Venins,Min_Venins,Moyenne_Glycanes,Max_Glycanes,Min_Glycanes
count,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,...,2351.0,2351.000000,2351.000000,2351.0,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000,2351.000000
mean,21.677584,6.405359,651.428328,11.150574,6.992344,0.523182,0.340281,0.452148,0.380264,0.041259,...,0.0,0.881073,13.238426,0.0,0.052087,0.162633,0.000094,0.544665,2.939137,0.000060
std,17.922809,3.634977,455.931904,2.200648,3.543520,0.499569,0.473904,0.497811,0.485555,0.198931,...,0.0,2.095343,26.090574,0.0,0.318630,0.917593,0.004537,2.274002,12.262601,0.002887
min,0.000000,1.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,3.000000,77.000000,10.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15.000000,6.000000,999.000000,11.000000,9.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.093409,2.220000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,32.000000,10.000000,999.000000,13.000000,9.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,0.0,0.805568,13.640000,0.0,0.000000,0.000000,0.000000,0.056667,0.305000,0.000000
max,102.000000,12.000000,999.000000,14.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.0,26.455000,227.440000,0.0,7.000000,17.900000,0.220000,41.395000,238.190000,0.140000


In [ ]:
V1.to_excel('ISAC_V1.xlsx', index=False)

##Repartition suivant chaque Target

In [ ]:
import pandas as pd
import plotly.express as px

value_counts = V1["Allergy_Present"].value_counts(normalize=True) * 100
df_plot = value_counts.reset_index()
df_plot.columns = ["Classe", "Pourcentage"]
df_plot["Classe"] = df_plot["Classe"].astype(str)

# Remplacer les valeurs 0/1 par des libellés explicites
classe_labels = {"0": "Sans allergie", "1": "Avec allergie"}
df_plot["Classe"] = df_plot["Classe"].map(classe_labels)

fig = px.pie(
    df_plot,
    names="Classe",
    values="Pourcentage",
    title="Répartition des classes dans 'Allergy_Present'",
    color="Classe",
    color_discrete_map={
        "Sans allergie": "#FF6F61",
        "Avec allergie": "#6B5B95"
    }
)

fig.update_traces(textinfo='percent+label', textfont_size=14)
fig.update_layout(
    title_font_size=15,
    width=800,
    height=400
)
fig.show()

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

allergy_positive = V1[V1["Allergy_Present"] == 1]

targets = ["Respiratory_Allergy", "Food_Allergy", "Venom_Allergy"]
labels_map = {"0": "Sans allergie", "1": "Avec allergie"}
colors_map = {
    "Sans allergie": "#FF6F61",
    "Avec allergie": "#6B5B95"
}

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}]*3],
                    subplot_titles=targets)

for i, col in enumerate(targets):
    counts = V1[col].value_counts(normalize=True) * 100
    df_plot = counts.reset_index()
    df_plot.columns = ["Classe", "Pourcentage"]
    df_plot["Classe"] = df_plot["Classe"].astype(str).map(labels_map)

    fig.add_trace(
        go.Pie(
            labels=df_plot["Classe"],
            values=df_plot["Pourcentage"],
            name=col,
            marker=dict(colors=[colors_map[label] for label in df_plot["Classe"]]),
            textinfo="percent+label"
        ),
        row=1, col=i+1
    )

fig.update_layout(
    title_text="Répartition des allergies par type",
    title_font_size=18,
    height=400,
    width=1000,
    showlegend=False
)

fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

res_pos = V1[V1["Respiratory_Allergy"] == 1]
food_pos = V1[V1["Food_Allergy"] == 1]
venom_pos = V1[V1["Venom_Allergy"] == 1]

columns = [
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Respiratory_Allergy_ARIA",
    "Type_of_Respiratory_Allergy_CONJ",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Gram",
    "Type_of_Respiratory_Allergy_GINA"
]

label_map = {"0": "Non allergique", "1": "Allergique"}
color_map = {"Non allergique": "#FF6F61", "Allergique": "#6B5B95"}

for col in columns:
    counts = res_pos[col].value_counts(normalize=True) * 100
    df_plot = counts.reset_index()
    df_plot.columns = ["Classe", "Pourcentage"]
    df_plot["Classe"] = df_plot["Classe"].astype(str).map(label_map)

    fig = px.pie(
        df_plot,
        names="Classe",
        values="Pourcentage",
        title=col,
        color="Classe",
        color_discrete_map=color_map
    )

    fig.update_traces(textinfo='percent+label', textfont_size=14)
    fig.update_layout(
        title_font_size=16,
        width=400,
        height=400,
        showlegend=True
    )
    fig.show()

In [ ]:
food_subtypes = [
    "Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts"
]


label_map = {"0": "Non allergique", "1": "Allergique"}
color_map = {"Non allergique": "#FF6F61", "Allergique": "#6B5B95"}

for col in food_subtypes:
    counts = food_pos[col].value_counts(normalize=True) * 100
    df_plot = counts.reset_index()
    df_plot.columns = ["Classe", "Pourcentage"]
    df_plot["Classe"] = df_plot["Classe"].astype(str).map(label_map)

    fig = px.pie(
        df_plot,
        names="Classe",
        values="Pourcentage",
        title=col,
        color="Classe",
        color_discrete_map=color_map
    )

    fig.update_traces(textinfo='percent+label', textfont_size=14)
    fig.update_layout(
        title_font_size=16,
        width=400,
        height=400,
        showlegend=True
    )
    fig.show()

##Quelques Graphes

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots

#repartition par age
fig1 = px.histogram(V1, x="Age", nbins=20, color_discrete_sequence=["skyblue"],
                    title="Répartition des âges")
fig1.update_layout(xaxis_title="Âge", yaxis_title="Nombre de personnes")

# prevelement par mois
df_month = V1["Blood_Month_sample"].dropna().astype(int).value_counts().sort_index().reset_index()
df_month.columns = ["Mois", "Nombre"]
fig2 = px.bar(df_month, x="Mois", y="Nombre", color_discrete_sequence=["lightcoral"],
              title="Nombre de prélèvements par mois")
fig2.update_layout(xaxis_title="Mois de prélèvement", yaxis_title="Nombre de personnes")

# region francaise
region_counts = V1['French_Region'].value_counts().reset_index()
region_counts.columns = ["Région", "Nombre"]
fig4 = px.bar(region_counts, x="Région", y="Nombre",
              color_discrete_sequence=["#2ca02c"],  # Vert
              title="Répartition par région française")
fig4.update_layout(xaxis_title="Région française",
                  yaxis_title="Nombre de personnes",
                  xaxis_tickangle=-45)

fig = make_subplots(rows=1, cols=3, subplot_titles=("Répartition des âges", "Prélèvements par mois", "Répartition par région française"),
                   shared_yaxes=False)

# Ajouter les sous-graphiques
for trace in fig1["data"]:
    fig.add_trace(trace, row=1, col=1)

for trace in fig2["data"]:
    fig.add_trace(trace, row=1, col=2)

for trace in fig4["data"]:
    fig.add_trace(trace, row=1, col=3)

# Mettre en page
fig.update_layout(
    height=500,
    width=1200,
    showlegend=False,
    margin=dict(r=50, l=50, b=100),  # Ajuster les marges pour les labels
    # Couleurs uniques pour chaque graphique
    colorway=["#1f77b4", "#ff7f0e", "#2ca02c"]
)


# Rotation des étiquettes sur l'axe x pour la région
fig.update_xaxes(tickangle=-45, row=1, col=3)

fig.show()

##PCA et UMAP

In [ ]:
targets = [
    "Allergy_Present",
    "Respiratory_Allergy",
    "Food_Allergy",
    "Venom_Allergy",
    "Severe_Allergy",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Herb",
    "Type_of_Respiratory_Allergy_IGE_Pollen_Tree",
    "Type_of_Respiratory_Allergy_IGE_Dander_Animals",
    "Type_of_Respiratory_Allergy_IGE_Mite_Cockroach",
    "Type_of_Respiratory_Allergy_IGE_Molds_Yeast",
    "Type_of_Food_Allergy_Aromatics",
    "Type_of_Food_Allergy_Other",
    "Type_of_Food_Allergy_Cereals_&_Seeds",
    "Type_of_Food_Allergy_Egg",
    "Type_of_Food_Allergy_Fish",
    "Type_of_Food_Allergy_Fruits_and_Vegetables",
    "Type_of_Food_Allergy_Mammalian_Milk",
    "Type_of_Food_Allergy_Oral_Syndrom",
    "Type_of_Food_Allergy_Other_Legumes",
    "Type_of_Food_Allergy_Peanut",
    "Type_of_Food_Allergy_Shellfish",
    "Type_of_Food_Allergy_TPO",
    "Type_of_Food_Allergy_Tree_Nuts",
    "Type_of_Venom_Allergy_ATCD_Venom",
    "Type_of_Venom_Allergy_IGE_Venom",

]

extra_columns = [
    "Chip_Type",
    "Chip_Code",
    ]

extra = ['History_of_food_anaphylaxis','First_degree_family_history_of_atopy', 'History_of_atopic_dematitis', 'History_of_mast_cell_disorders',
         'History_of_hymenoptera_venom_anaphylaxis','Mammalian_meat']

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

X= V1.copy()
X.drop(targets, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)

X = X.iloc[:, 1:]

y = V1["Allergy_Present"]


# 3. Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Reconstruction d’un DataFrame avec les résultats PCA
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_pca["Allergy_Present"] = y.values  # Ajouter la cible

# Tracé du graphique
fig = px.scatter(
    df_pca,
    x="PC1",
    y="PC2",
    color=df_pca["Allergy_Present"].astype(str),  # couleur par classe 0 ou 1
    title="Projection PCA des patients",
    labels={
        "PC1": "Première composante principale",
        "PC2": "Deuxième composante principale",
        "color": "Présence d'allergie"
    },
    color_discrete_map={"0": "blue", "1": "red"},
    width=900,
    height=650
)

fig.update_layout(
    legend_title="Présence d'allergie"
)

fig.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import plotly.express as px

# 1. Préparation des données
X = V1.copy()
X.drop(targets, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)
X = X.iloc[:, 1:]

y = V1["Allergy_Present"]

# 2. Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3. PCA en 3D
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

# 4. Reconstruction du DataFrame
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2", "PC3"])
df_pca["Allergy_Present"] = y.values

# 5. Affichage graphique en 3D
fig = px.scatter_3d(
    df_pca,
    x="PC1",
    y="PC2",
    z="PC3",
    color=df_pca["Allergy_Present"].astype(str),
    title="Projection PCA en 3D des patients",
    labels={"color": "Présence d'allergie"},
    color_discrete_map={"0": "blue", "1": "red"},
    width=950,
    height=700
)

fig.update_layout(legend_title="Présence d'allergie")
fig.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

X= V1.copy()
X.drop(targets, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)

X = X.iloc[:, 1:]

y = V1["Respiratory_Allergy"]


# 3. Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Reconstruction d’un DataFrame avec les résultats PCA
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_pca["Respiratory_Allergy"] = y.values  # Ajouter la cible

# Tracé du graphique
fig = px.scatter(
    df_pca,
    x="PC1",
    y="PC2",
    color=df_pca["Respiratory_Allergy"].astype(str),  # couleur par classe 0 ou 1
    title="Projection PCA des patients",
    labels={
        "PC1": "Première composante principale",
        "PC2": "Deuxième composante principale",
        "color": "Présence d'allergie"
    },
    color_discrete_map={"0": "blue", "1": "red"},
    width=900,
    height=650
)

fig.update_layout(
    legend_title="Présence d'allergie"
)

fig.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

V1_aller = V1[V1["Allergy_Present"] == 1]
X= V1_aller.copy()
X.drop(targets, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)

X = X.iloc[:, 1:]

y = V1_aller["Severe_Allergy"]


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_pca["Severe_Allergy"] = y.values  # Ajouter la cible

fig = px.scatter(
    df_pca,
    x="PC1",
    y="PC2",
    color=df_pca["Severe_Allergy"].astype(str),  # couleur par classe 0 ou 1
    title="Projection PCA des patients",
    labels={
        "PC1": "Première composante principale",
        "PC2": "Deuxième composante principale",
        "color": "Severe d'allergie"
    },
    color_discrete_map={"0": "blue", "1": "red"},
    width=900,
    height=650
)

fig.update_layout(
    legend_title="Présence d'allergie"
)
fig.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

V1_aller = V1[V1["Allergy_Present"] == 1]
X= V1_aller.copy()
X.drop(targets, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)

X = X.iloc[:, 1:]

y = V1_aller["Food_Allergy"]


# 3. Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Reconstruction d’un DataFrame avec les résultats PCA
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_pca["Food_Allergy"] = y.values  # Ajouter la cible

# Tracé du graphique
fig = px.scatter(
    df_pca,
    x="PC1",
    y="PC2",
    color=df_pca["Food_Allergy"].astype(str),  # couleur par classe 0 ou 1
    title="Projection PCA des patients",
    labels={
        "PC1": "Première composante principale",
        "PC2": "Deuxième composante principale",
        "color": "Food d'allergie"
    },
    color_discrete_map={"0": "blue", "1": "red"},
    width=900,
    height=650
)


fig.update_layout(
    legend_title="Food Allergie"
)

fig.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

V1_aller = V1[V1["Allergy_Present"] == 1]
X= V1_aller.copy()
X.drop(targets, axis=1, inplace=True)
X.drop(extra_columns, axis=1, inplace=True)
X.drop(extra, axis=1, inplace=True)

X = X.iloc[:, 1:]

y = V1_aller["Venom_Allergy"]


# 3. Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Reconstruction d’un DataFrame avec les résultats PCA
df_pca = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
df_pca["Venom_Allergy"] = y.values  # Ajouter la cible

# Tracé du graphique
fig = px.scatter(
    df_pca,
    x="PC1",
    y="PC2",
    color=df_pca["Venom_Allergy"].astype(str),  # couleur par classe 0 ou 1
    title="Projection PCA des patients",
    labels={
        "PC1": "Première composante principale",
        "PC2": "Deuxième composante principale",
        "color": "Venom d'allergie"
    },
    color_discrete_map={"0": "blue", "1": "red"},
    width=900,
    height=650
)


fig.update_layout(
    legend_title="Venom Allergie"
)

fig.show()